In [55]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
import json
import string
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import casual, casual_tokenize
from nltk import bigrams

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [3]:
consumer_key = 'WxNCHN3AZKIbby85LE0FRn52h'
consumer_secret = 'sWuF83Pianv1Vm9R0txN34DROWFzJGpUZfQz673RytxhRRwEtL'
access_token = '975767174-qFHaY394JvFX8DbgKmq2cSQjsl1Cxhl1AuggDokq'
access_secret = '5eV3SHTA0HU7i7USSwxpspoqSk2sW6Ip7VBXqOgMLxsF9'

In [4]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token,access_secret)
api = tweepy.API(auth)

In [5]:
#Reading Own Timeline
for status in tweepy.Cursor(api.home_timeline).items(2):
    print(status.text)

RT @IndianNazi: #AdarshLiberal  i dont say u r bad or good, just answer one question if u have guts &amp; all will know truth  - why u want to…
#INDvENG @YUVSTRONG12, Rishabh Pant in @imVkohli-led squads https://t.co/dnjLRe3jPv via @TOISportsNews https://t.co/llUF4W81lE


In [6]:
#Reading own timeline-- JSON
for status in tweepy.Cursor(api.home_timeline).items(1):
    print(status._json)

{'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'coordinates': None, 'favorite_count': 0, 'is_quote_status': False, 'text': 'RT @IndianNazi: #AdarshLiberal  i dont say u r bad or good, just answer one question if u have guts &amp; all will know truth  - why u want to…', 'id_str': '817332335003729920', 'in_reply_to_user_id_str': None, 'created_at': 'Fri Jan 06 11:29:48 +0000 2017', 'retweet_count': 6, 'geo': None, 'in_reply_to_status_id_str': None, 'retweeted': False, 'favorited': False, 'in_reply_to_user_id': None, 'entities': {'hashtags': [{'indices': [16, 30], 'text': 'AdarshLiberal'}], 'user_mentions': [{'indices': [3, 14], 'screen_name': 'IndianNazi', 'id_str': '3071679078', 'name': 'Indian Nazi', 'id': 3071679078}], 'urls': [], 'symbols': []}, 'in_reply_to_screen_name': None, 'id': 817332335003729920, 'user': {'profile_banner_url': 'https://pbs.twimg.com/profile_banners/3071679078/1439984477', 'description': 'Jinhe naz hai India par', 'contributors

In [7]:
#Streaming (All the tweets with #Kaggle hashtag:

class MyListener(StreamListener):
    def __init__(self, time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        self.saveFile = open('python.json', 'a')
        super(MyListener, self).__init__()

    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            self.saveFile.write(data)
            return True
        else:
            self.saveFile.close()
            return False

In [22]:
twitter_stream = Stream(auth, MyListener(time_limit = 20))
twitter_stream.filter(track=['Trump'])

In [23]:
#Body of a tweet-- Preprocessing
with open('python.json','r') as f:
    line = f.readline()
    tweet = json.loads(line)
    print (json.dumps(tweet, indent=3))

{
   "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
   "coordinates": null,
   "favorite_count": 0,
   "is_quote_status": true,
   "text": "@Evan_McMullin @ananavarro   Shouldn't the man elected President care about the United States of America? https://t.co/Qr80dG63we",
   "id_str": "817333854738399232",
   "in_reply_to_user_id_str": "1051396218",
   "created_at": "Fri Jan 06 11:35:50 +0000 2017",
   "user": {
      "profile_banner_url": "https://pbs.twimg.com/profile_banners/777671141452238848/1483320930",
      "description": "#NeverTrump #NoDAPL TINN & WSH. Cussing is ok but will block locker room talk. Hunting for sport is chickenshit. I don't coddle.",
      "contributors_enabled": false,
      "following": null,
      "favourites_count": 7432,
      "utc_offset": null,
      "id_str": "777671141452238848",
      "listed_count": 0,
      "created_at": "Mon Sep 19 00:50:23 +0000 2016",
      "profile_background_color": "F5F8FA

In [24]:
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
tk = TweetTokenizer()
tk.tokenize(tweet)

['RT',
 '@marcobonzanini',
 ':',
 'just',
 'an',
 'example',
 '!',
 ':D',
 'http://example.com',
 '#NLP']

In [49]:
st = ''
with open('python.json','r') as f:
    for line in f:
        tweet = json.loads(line)
        print (tk.tokenize(tweet['text']))
        st += ' '.join(tk.tokenize(tweet['text']))

['@Evan_McMullin', '@ananavarro', "Shouldn't", 'the', 'man', 'elected', 'President', 'care', 'about', 'the', 'United', 'States', 'of', 'America', '?', 'https://t.co/Qr80dG63we']
['RT', '@ezlusztig', ':', "Here's", 'what', 'I', 'want', 'to', 'know', ':', 'What', 'will', 'they', 'tell', 'him', 'they', 'know', 'about', 'HIS', 'OWN', 'collusion', 'with', 'Russian', 'hackers', '?', 'And', 'is', '…']
['LMAOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO', 'https://t.co/QOgrC5C35W']
['RT', '@NBCNews', ':', 'Ex-CIA', 'director', 'James', 'Woolsey', 'quits', 'Trump', 'transition', 'team', 'https://t.co/9WML9Qlc52', 'https://t.co/4AMK2dSX5U']
['Toyota', 'à', 'son', 'tour', 'dans', 'le', 'viseur', 'de', 'Donald', 'Trump', 'https://t.co/Xgq1SnD2nz']
['what', 'a', 'dumbass', 'bitch', 'https://t.co/81hYINHmPg']
['RT', '@MightyBusterBro', ':', '.', 'HOLLYWOOD', '"', 'TRUMPS', '"', 'B', '.', 'L', '.', 'M', '.', 'Democrat', 'Sore', 'Losers', 'vs', 'Trump-Hating', 'Racists', '#BLMKidnapping', '#racism', '#crime', '…']


In [50]:
#Counting Occurences 
with open('python.json', 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        terms = [term for term in tk.tokenize(tweet['text'])]
        count_all.update(terms)
    print (count_all.most_common(5))

[(':', 178), ('Trump', 143), ('RT', 126), ('.', 107), ('to', 83)]


In [30]:
""" Since the above results are not interesting we need to remove stopwords: """
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['RT', 'via', '…', 'I', "Trump's",'...','trump','de']


In [51]:
with open('python.json', 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        terms = [term for term in tk.tokenize(tweet['text']) if term not in stop]
        count_all.update(terms)
    print (count_all.most_common(20))
    st += ' '.join(terms)

[('Trump', 143), ('Donald', 19), ('ask', 18), ("don't", 15), ('look', 15), ('get', 15), ("I'm", 15), ('face', 15), ('going', 14), ('blows', 14), ('😈', 14), ('@Perry0667', 14), ('surprised', 14), ('world', 14), ('Botox', 14), ('doctor', 14), ('’', 14), ('#Trump', 12), ('Obama', 11), ('Toyota', 10)]


In [45]:
with open('python.json', 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        terms = [term for term in tk.tokenize(tweet['text']) if term.startswith('#')]
        
        count_all.update(terms)
    print (count_all.most_common(10))

[('#Trump', 12), ('#MAGA', 3), ('#Obamacare', 3), ('#Maga', 2), ('#NiggerNavy', 2), ('#ElectoralCollege', 2), ('#trump', 2), ('#BLMKidnapping', 2), ('#niggernavy', 2), ('#draintheswamp', 2)]


In [ ]:
# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(st)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()